In [12]:
import os
import re
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import torch
import torchvision.transforms.functional as TF
import tifffile
import yaml
from einops import rearrange


In [13]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
%autoreload 2

In [15]:
from mushroom.data.multiplex import extract_ome_tiff, write_basic_ome_tiff, get_ome_tiff_channels, make_pseudo
from mushroom.data.visium import adata_from_visium
from mushroom.visualization.utils import display_sections
import mushroom.utils as utils

In [16]:
run_dir = '/data/estorrs/mushroom/data/projects/htan_talk'
mushroom_dir = os.path.join(run_dir, 'mushroom')
Path(mushroom_dir).mkdir(parents=True, exist_ok=True)

In [17]:
metadata = yaml.safe_load(open(os.path.join(run_dir, 'registered', 'metadata_v2.yaml')))
metadata

{'HT225C1': {'data': {'visium': {'HT225C1-U2': '/diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT225C1/H1/HT225C1-Th1Fc1U2Z1B1/outs',
    'HT225C1-U3': '/diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT225C1/H1/HT225C1-Th1Fc1U3Z1B1/outs',
    'HT225C1-U4': '/diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT225C1/H1/HT225C1-Th1Fc1U4Z1B1/outs',
    'HT225C1-U5': '/diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT225C1/H1/HT225C1-Th1Fc1U5Z1B1/outs'}},
  'ids': ['s0', 's1', 's2', 's3'],
  'order': ['HT225C1-U2', 'HT225C1-U3', 'HT225C1-U4', 'HT225C1-U5'],
  'registered_pixels_per_micron': 2.484081102305233,
  'scale': 0.2},
 'HT413C1_Th1k4A1': {'data': {'he': {'HT413C1_Th1k4A1_U1': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/htan_talk/registered/HT413C1_Th1k4A1/s4_he.tif',
    'HT413C1_Th1k4A1_U11': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/htan_talk/registered/HT413C1_Th1k4A1/s10_he.tif',
    'HT413C1_Th1k4A1_U14':

In [18]:
configs = {}
for case, data in metadata.items():
    config = [
        {
            'id': sid,
            'position': 0,
            'data': []
        }
        for sid in data['ids']
    ]
    
    for dtype, mapping in data['data'].items():
        for sample_id, filepath in mapping.items():
            idx = data['order'].index(sample_id)
            
            # convert from katmai to glacier
            filepath = filepath.replace('/diskmnt/Projects/Users/estorrs', '/data/estorrs')
            
            config[idx]['data'].append(
                {
                    'dtype': dtype,
                    'filepath': filepath,
                }
            )
    configs[case] = config
configs
    

{'HT225C1': [{'id': 's0',
   'position': 0,
   'data': [{'dtype': 'visium',
     'filepath': '/diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT225C1/H1/HT225C1-Th1Fc1U2Z1B1/outs'}]},
  {'id': 's1',
   'position': 0,
   'data': [{'dtype': 'visium',
     'filepath': '/diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT225C1/H1/HT225C1-Th1Fc1U3Z1B1/outs'}]},
  {'id': 's2',
   'position': 0,
   'data': [{'dtype': 'visium',
     'filepath': '/diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT225C1/H1/HT225C1-Th1Fc1U4Z1B1/outs'}]},
  {'id': 's3',
   'position': 0,
   'data': [{'dtype': 'visium',
     'filepath': '/diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT225C1/H1/HT225C1-Th1Fc1U5Z1B1/outs'}]}],
 'HT413C1_Th1k4A1': [{'id': 's0',
   'position': 0,
   'data': [{'dtype': 'he',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/HT413C1_Th1k4A1/s0_he.tif'}]},
  {'id': 's1',
   'position': 0,
   'data': [{'dtype': 'he',


In [19]:
import sys
yaml.safe_dump(configs, sys.stdout)

HT225C1:
- data:
  - dtype: visium
    filepath: /diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT225C1/H1/HT225C1-Th1Fc1U2Z1B1/outs
  id: s0
  position: 0
- data:
  - dtype: visium
    filepath: /diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT225C1/H1/HT225C1-Th1Fc1U3Z1B1/outs
  id: s1
  position: 0
- data:
  - dtype: visium
    filepath: /diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT225C1/H1/HT225C1-Th1Fc1U4Z1B1/outs
  id: s2
  position: 0
- data:
  - dtype: visium
    filepath: /diskmnt/Datasets/Spatial_Transcriptomics/outputs_OCT/Human/HT225C1/H1/HT225C1-Th1Fc1U5Z1B1/outs
  id: s3
  position: 0
HT413C1_Th1k4A1:
- data:
  - dtype: he
    filepath: /data/estorrs/mushroom/data/projects/htan_talk/registered/HT413C1_Th1k4A1/s0_he.tif
  id: s0
  position: 0
- data:
  - dtype: he
    filepath: /data/estorrs/mushroom/data/projects/htan_talk/registered/HT413C1_Th1k4A1/s1_he.tif
  id: s1
  position: 0
- data:
  - dtype: xenium
    filepath: /data/es

In [19]:
# # filling out position seperately and reading back in
# yaml.safe_dump(configs, open(os.path.join(mushroom_dir, 'configs_template.yaml'), 'w'))

In [20]:
configs = yaml.safe_load(open(os.path.join(mushroom_dir, 'configs_template.yaml')))
configs

{'S18-9906': [{'data': [{'dtype': 'he',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s0_he.tif'},
    {'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s0_visium.h5ad'}],
   'id': 's0',
   'position': 0},
  {'data': [{'dtype': 'xenium',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s1_xenium.h5ad'}],
   'id': 's1',
   'position': 5},
  {'data': [{'dtype': 'multiplex',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s2_multiplex.ome.tiff'}],
   'id': 's2',
   'position': 10},
  {'data': [{'dtype': 'xenium',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s3_xenium.h5ad'}],
   'id': 's3',
   'position': 40},
  {'data': [{'dtype': 'multiplex',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s4_multiplex.ome.tiff'}],
   'id': 's4',
   'position': 45

In [21]:
for case, config in configs.items():
    directory = os.path.join(mushroom_dir, case)
    Path(directory).mkdir(parents=True, exist_ok=True)
    
    yaml.safe_dump(config, open(os.path.join(directory, 'config.yaml'), 'w'))

In [22]:
mushroom_dir

'/data/estorrs/mushroom/data/projects/htan_talk/mushroom'